In [78]:
import json
import jupyter_dash
import urllib.request 
import dash_core_components as dcc 
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_cytoscape as cyto 

import pandas as pd 
import numpy as np 

import networkx as nx 

import plotly.express as px 

from IPython.display import Javascript 

In [19]:
df = pd.read_csv("./data_csv/table.csv") 
df.head() 

,name_x,name_y,count
0,scrambled eggs,salt mackerel,93
1,apple tartlets,stuffed olives,84
2,prunelle brandy,apple tartlets,51
3,prunelle brandy,stuffed olives,50
4,cracked wheat,scrambled eggs,40


In [184]:
df_year = pd.read_csv("./data_csv/scatter_data.csv").sort_values(by = "year") 
df_year.head() 

,name_x,year,count
248,blackberry pie,1854.0,1
171,highland punch,1856.0,1
170,scrambled eggs,1856.0,2
212,codfish balls,1856.0,1
193,"omelet, plain",1858.0,1


In [28]:
dish_names = df["name_x"] 
options = [] 
for name in dish_names: 
    options.append({
        "label": name, 
        "value": name 
    }) 

In [41]:
def cytoscape_data(name): 
    G = nx.Graph() 
    dff = df[df["name_x"] == name] 
    for name in dff["name_x"]: 
        G.add_node(
            name, 
            color = "lightblue" 
        ) 
    for index in range(len(dff.index)): 
        G.add_edge(
            dff["name_x"].tolist()[index], 
            dff["name_y"].tolist()[index], 
            weight = dff["count"].tolist()[index]
        ) 
    
    nx_data = nx.cytoscape_data(G) 
    return nx_data["elements"] 

In [14]:
with open("food.json", "r") as file: 
    data = json.loads(file.read()) 

In [229]:
app = jupyter_dash.JupyterDash(__name__)
app.title = "Food Graph"
server = app.server

app.scripts.config.serve_locally = True
app.css.config.serve_locally = True


# with open('heroku_application/food.json', 'r') as f:
    # data = json.loads(f.read()) 
  
# https://github.com/plotly/dash-cytoscape/blob/master/demos/data/edge-types/cy-style.json
with open('heroku_application/cy-style.json') as f:
    stylesheet = json.loads(f.read())

styles = {
    'container': {
        'position': 'fixed',
        'display': 'flex',
        'flex-direction': 'column',
        'height': '100%',
        'width': '100%', 
        "float": "left"
    },
    'cy-container': {
        'flex': '4',
        'position': 'relative'
    },
    'cytoscape': {
        'position': 'absolute',
        "x": "0px", 
        "y": "0px", 
        'width': '100%',
        'height': '100%',
        'z-index': 999
    }, 
}


# App
app.layout = html.Div([ 
    html.H1("What's on the menu? Let's give a look!"), 
    html.Div([ 
        html.H4("A specific dish? Choose here!"), 
        dcc.Dropdown( 
            id = "demo-dropdown", 
            options = [{"label": "Every food!", "value": "null"}] + options
        ), 
        # dcc.Graph(id = "barplot") 
    ], style = {"width": "32%", "display": "inline-block", "float": "right"}), 
    html.Div(className='cy-container', style=styles['cy-container'], children=[
        cyto.Cytoscape(
            id='cytoscape',
            elements=data['elements'],
            stylesheet=stylesheet,
            style=styles['cytoscape'],
            layout={
                'name': 'cose',
                'idealEdgeLength': 100,
                'nodeOverlap': 10,
                'refresh': 20,
                'fit': True,
                'padding': 200,
                'randomize': False,
                'componentSpacing': 100,
                'nodeRepulsion': 4000000,
                'edgeElasticity': 50,
                'nestingFactor': 5,
                'gravity': 80,
                'numIter': 1000,
                'initialTemp': 200,
                'coolingFactor': 0.95,
                'minTemp': 1.0,
                'minNodeSpacing': 100
            },
            responsive=True 
        ), 
    ]), 
    html.Div([ 
        html.Div([
            dcc.Graph(id = "barplot", figure = px.bar(height = 358)) 
        ], style = {"width": "38%", "float": "left", "display": "inline-block"}), 
        html.Div([ 
            html.Div(id = "dish"), 
            html.Div(id = "dish_year") 
        ], style = {"width": "16%", "float": "left", "display": "inline-block", 
                    "margin-top": 49, "font-family": "Serif", "font-size": 25}), 
        html.Div([
            dcc.Graph(id = "lineplot", figure = px.line(height = 358)) 
        ], style = {"width": "38%", "float": "left", "display": "inline-block"}), 
    ], style = {"width": "100%", "height": "34%"}), 
], style=styles['container'])

@app.callback(Output(component_id = "cytoscape", component_property = "elements"), 
             [Input(component_id = "demo-dropdown", component_property = "value")]) 
def display_graph(value): 
    if value is not None and value != "null": 
        return cytoscape_data(value) 
    else: 
        return data["elements"] 

@app.callback(Output(component_id = "barplot", component_property = "figure"), 
             [Input(component_id = "cytoscape", component_property = "tapNodeData")]) 
def display_barplot(node): 
    name = node["id"] if node is not None else "bar" 
    if name == "bar": 
        return px.bar(height = 358)  
    df_node = df[df["name_x"] == name] 
    print(name) 
    fig = px.bar( 
        df_node, 
        x = "name_y", 
        y = "count", 
        height = 358 
    ) 
    fig.update_layout(
        xaxis_title = "Dish's name", 
        yaxis_title = "Count" 
    ) 
    return fig 

@app.callback(Output(component_id = "lineplot", component_property = "figure"), 
             [Input(component_id = "cytoscape", component_property = "tapNodeData")]) 
def display_lineplot(node): 
    name = node["id"] if node is not None else "line" 
    if name == "line": 
        return px.line(height = 358) 
    df_year_node = df_year[df_year["name_x"] == name] 
    fig = px.line(
        df_year_node, 
        x = "year", 
        y = "count" 
    ) 
    fig.update_layout(
        xaxis_title = "Year", 
        yaxis_title = "Count" 
    ) 
    return fig 

@app.callback(Output(component_id = "dish", component_property = "children"),  
             [Input(component_id = "cytoscape", component_property = "tapNodeData")]) 
def display_text(node): 
    if node is None: 
        return "Select a node!" 
    else: 
        return "That's the dish you choose: {dish}. I love it!".format(dish = node["id"]) 


@app.callback(Output(component_id = "dish_year", component_property = "children"), 
             [Input(component_id = "cytoscape", component_property = "tapNodeData")]) 
def display_dish_text(node): 
    if node is None: 
        return "A blue node!" 
    else: 
        df_dish_node = df[df["name_x"] == node["id"]] 
        n = df_dish_node["count"].sum() 
        return "It appears on {n} menu{s}!".format(n = n,)

if __name__ == "__main__": 
    app.run_server(debug = True,  port = 9999) 

Dash app running on http://127.0.0.1:9999/
chocolate cuts
soft clam stew
italian salad
sliced tomatoe
